## Load dataset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv('google-landmarks-dataset/train.csv')
test_data = pd.read_csv('google-landmarks-dataset/test.csv')

In [ ]:
train_data.head(5)

## Sample the data

In [ ]:
landmark_list = [str(x) for x in range(1000, 3001)]
train_data_sample = train_data[train_data['landmark_id'].isin(landmark_list)]

In [ ]:
print('sample train data:', len(train_data_sample))